In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('BigBasket Products.csv',index_col='index')

In [3]:
df.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [4]:
df.drop_duplicates(inplace = True, subset=['product'])

In [5]:
df.head()
save_df = df.copy()

In [6]:
print('category values')
df.category.value_counts()

category values


Beauty & Hygiene            6825
Gourmet & World Food        4040
Kitchen, Garden & Pets      3185
Cleaning & Household        2408
Snacks & Branded Foods      2401
Foodgrains, Oil & Masala    1964
Bakery, Cakes & Dairy        746
Beverages                    740
Baby Care                    544
Fruits & Vegetables          350
Eggs, Meat & Fish            338
Name: category, dtype: int64

In [7]:
print('subcategory values')
df.sub_category.value_counts()

subcategory values


Skin Care                   2079
Health & Medicine           1019
Storage & Accessories        875
Fragrances & Deos            853
Bath & Hand Wash             852
                            ... 
Herbs & Seasonings            16
Water                          7
Flower Bouquets, Bunches       7
Pork & Other Meats             3
Marinades                      1
Name: sub_category, Length: 90, dtype: int64

# Popularity based filtering

Popularity-based technique use the idea of recommend popular items, so I will recommend high rating items descendingPopularity-based technique use the idea of recommend popular items, so I will recommend high rating items descending

On the website where new user visit, I will show each top 5 items of each categories. So, the user can explore what currently trending or popular.

In [8]:
# Group the DataFrame by category, and sort by rating
grouped = df.sort_values(by='rating', ascending=False).groupby('category')

# Define a function to return the top 5 products from each group
def top_5(group):
    return group.head(5)

# Apply the function to each group, and concatenate the results
result = grouped.apply(top_5).reset_index(drop=True)

# Display the resulting DataFrame
print(result[['category', 'sub_category', 'product', 'rating']])

                    category              sub_category  \
0                  Baby Care           Diapers & Wipes   
1                  Baby Care          Baby Accessories   
2                  Baby Care       Baby Bath & Hygiene   
3                  Baby Care           Diapers & Wipes   
4                  Baby Care           Diapers & Wipes   
5      Bakery, Cakes & Dairy                     Dairy   
6      Bakery, Cakes & Dairy     Cookies, Rusk & Khari   
7      Bakery, Cakes & Dairy          Cakes & Pastries   
8      Bakery, Cakes & Dairy          Cakes & Pastries   
9      Bakery, Cakes & Dairy                     Dairy   
10          Beauty & Hygiene                 Skin Care   
11          Beauty & Hygiene                 Skin Care   
12          Beauty & Hygiene         Health & Medicine   
13          Beauty & Hygiene         Fragrances & Deos   
14          Beauty & Hygiene          Feminine Hygiene   
15                 Beverages                       Tea   
16            

# Keyword-based filtering

Let users to enter one or more keywords related to the products they are interested in, and it returns a list of items that match those keywords

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the 'description' column
tfidf_matrix = vectorizer.fit_transform(df['description'].values.astype('U'))

def recommend_product(input_text):
    # Transform the input text using the vectorizer
    input_vector = vectorizer.transform([input_text])

    # Calculate the cosine similarity between input and dataset
    similarities = cosine_similarity(input_vector, tfidf_matrix)

    # Get the index of the most similar product
    most_similar_index = similarities.argmax()

    # Return the details of the most similar product
    most_similar_product = df.iloc[most_similar_index]
    return most_similar_product

# Test the function
user_input = input("Enter your text: ")
recommended_product = recommend_product(user_input)
print("Recommended Product:")
print("Name:", recommended_product['product'])
print("Brand:", recommended_product['brand'])
print("Category:", recommended_product['category'])

Enter your text: Baby
Recommended Product:
Name: Babycare - Gift Pack
Brand: Himalaya Baby
Category: Baby Care


In [11]:
def recommend_products(input_text, num_recommendations=5):
    # Transform the input text using the vectorizer
    input_vector = vectorizer.transform([input_text])

    # Calculate the cosine similarity between input and dataset
    similarities = cosine_similarity(input_vector, tfidf_matrix)

    # Get the indices of the top N most similar products
    #The argsort function is used to get the indices of the top num_recommendations most similar products. 
    top_indices = similarities.argsort()[0][-num_recommendations:][::-1]

    # Return the details of the top N most similar products
    top_products = df.iloc[top_indices]
    return top_products
# Test the function
user_input = input("Enter your text: ")
recommended_products = recommend_products(user_input)
print("Recommended Products:")
for index, product in recommended_products.iterrows():
    print("Name:", product['product'])
    print("Brand:", product['brand'])
    print("Category:", product['category'])
    print("-----------------------------")

Enter your text: Stationary
Recommended Products:
Name: Pencil Cup - Deluxe, No.1704
Brand: Omega
Category: Cleaning & Household
-----------------------------
Name: Pen Stand - Hexa, No.1705
Brand: Omega
Category: Cleaning & Household
-----------------------------
Name: Pen Holder - Elite, No.1727
Brand: Omega
Category: Cleaning & Household
-----------------------------
Name: Stapler 10
Brand: Kangaroo
Category: Cleaning & Household
-----------------------------
Name: Modular Multi Utility Plastic Storage Drawer - 3 Tier, Assorted Colour
Brand: Nakoda
Category: Kitchen, Garden & Pets
-----------------------------


In [12]:
def recommend_products(input_text, num_recommendations=5):
    # Transform the input text using the vectorizer
    input_vector = vectorizer.transform([input_text])

    # Calculate the cosine similarity between input and dataset
    similarities = cosine_similarity(input_vector, tfidf_matrix)

    # Get the indices of the top N most similar products
    #The argsort function is used to get the indices of the top num_recommendations most similar products. 
    top_indices = similarities.argsort()[0][-num_recommendations:][::-1]

    # Return the details of the top N most similar products
    top_products = df.iloc[top_indices]
    return top_products
# Test the function
user_input = input("Enter your text: ")
recommended_products = recommend_products(user_input)
print("Recommended Products:")
for index, product in recommended_products.iterrows():
    print("Name:", product['product'])
    print("Brand:", product['brand'])
    print("Category:", product['category'])
    print("-----------------------------")

Enter your text: Garlic
Recommended Products:
Name: Powder - Garlic
Brand: Organic Nation
Category: Foodgrains, Oil & Masala
-----------------------------
Name: Select - Garlic Powder
Brand: Valley Spice
Category: Foodgrains, Oil & Masala
-----------------------------
Name: Organic Garlic Powder - Dehydrated
Brand: bb Royal
Category: Foodgrains, Oil & Masala
-----------------------------
Name: Strong Garlic Instant Noodles
Brand: A&M
Category: Snacks & Branded Foods
-----------------------------
Name: Pickle - Ginger
Brand: Priya
Category: Snacks & Branded Foods
-----------------------------
